<a href="https://colab.research.google.com/github/madhuri1503/MOVIE-RECOMMENDER-SYSTEM/blob/main/MOVIE_RECOMMENDERSYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOVIE RECOMMENDERSYSTEM



Import libraries and load dataset

In [ ]:
import pandas as pd
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
movies=pd.read_csv('/content/drive/MyDrive/edureka/movies_csv_file_for _final_project.csv')
ratings=pd.read_csv('/content/drive/MyDrive/edureka/ratings.csv')

In [ ]:
movies_df=pd.DataFrame(movies)
ratings_df=pd.DataFrame(ratings)

In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


Exploratory Data Analysis including:
Understanding of distribution of the features available.

Finding unique users and movies.

Average rating and Total movies at genre level.

Unique genres considered..

In [ ]:
# Understanding the distribution of the features available in the movies dataset
movies_df.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
# Finding unique users and movies
unique_users = ratings_df["userId"].nunique()
unique_movies = movies_df["movieId"].nunique()
print("Unique Users:", unique_users)
print("Unique Movies:", unique_movies)



Unique Users: 668
Unique Movies: 10329


In [ ]:
# Check if the genres column exists in the movies dataframe
if "genres" not in movies_df.columns:
    print("The 'genres' column does not exist in the movies dataframe.")
else:
    print("The 'genres' column exists in the movies dataframe.")


The 'genres' column exists in the movies dataframe.


In [ ]:
# Split the genres column into separate columns
movies_df = movies_df.join(movies_df["genres"].str.split("|", expand=True))


In [ ]:
# Average rating and Total movies at genre level
merged_df = pd.merge(movies_df, ratings_df, on="movieId", how="left")
grouped_df = merged_df.groupby(["genres"]).agg({"rating": "mean", "movieId": "count"})
grouped_df.head()


,rating,movieId
genres,,
(no genres listed),3.071429,7
Action,2.836406,217
Action|Adventure,3.739804,613
Action|Adventure|Animation,4.125000,12
Action|Adventure|Animation|Children,3.550000,10


In [ ]:
# Unique genres considered
unique_genres = movies_df["genres"].nunique()
print("Unique Genres:", unique_genres)

Unique Genres: 938


Create a popularity-basedrecommender system at a genre level. The userwill input a genre (g), minimum rating threshold (t) for a movie,and number of recommendations(N) for which it should be recommended top N movies which are most popular within that genre (g) ordered by ratings in descending order where each movie has at least (t) reviews

.Example:

Input:

•Genre (g) : |Fantasy

•Minimum reviews threshold(t):80

•Num recommendations (N) : 5

In [ ]:
import pandas as pd

# Get inputs from the user
genre = input("Enter genre (e.g. Comedy): ")
min_reviews = int(input("Enter minimum number of reviews: "))
num_recs = int(input("Enter number of recommendations: "))

# Filter the movies dataframe by genre and rating threshold
genre_movies = movies_df[movies_df["genres"].str.contains(genre)]
movie_ratings = pd.DataFrame(ratings_df.groupby("movieId")["rating"].mean())
movie_ratings["num_reviews"] = ratings_df.groupby("movieId")["rating"].count()
movie_ratings = movie_ratings[movie_ratings["num_reviews"] >= min_reviews]
genre_movies = genre_movies.merge(movie_ratings, on="movieId")
genre_movies = genre_movies.sort_values(by=["rating"], ascending=False)

# Output the top N recommended movies
print("\nTop {} recommended {} movies:".format(num_recs, genre))
print(genre_movies[["title", "rating", "num_reviews"]].head(num_recs))


Enter genre (e.g. Comedy): Fantasy
Enter minimum number of reviews: 80
Enter number of recommendations: 5

Top 5 recommended Fantasy movies:
                                                title    rating  num_reviews
11             Monty Python and the Holy Grail (1975)  4.301948          154
12                         Princess Bride, The (1987)  4.163743          171
25  Lord of the Rings: The Fellowship of the Ring,...  4.126984          189
28  Lord of the Rings: The Return of the King, The...  4.108434          166
26      Lord of the Rings: The Two Towers, The (2002)  4.095808          167


Create a content-based recommender system thatrecommends top N movies based on similar movie(m) genres

.Example:

Input:

•Movie Title (t): Toy Story

•Num recommendations (N): 5

In [ ]:
import pandas as pd

# Load the movies and ratings data into pandas dataframes
movies_df = pd.read_csv('/content/drive/MyDrive/edureka/movies_csv_file_for _final_project.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/edureka/ratings.csv')

# Pre-process the movies data to extract genre information
genres = []
for genre in movies_df["genres"]:
    genres.extend(genre.split("|"))
genres = set(genres)

movie_to_genres = {}
for movie, row in movies_df.iterrows():
    movie_genres = row["genres"].split("|")
    movie_to_genres[row["title"]] = movie_genres

# Calculate the similarity between movies based on genre information
def jaccard_similarity(movie1, movie2):
    set1 = set(movie_to_genres[movie1])
    set2 = set(movie_to_genres[movie2])
    return len(set1 & set2) / len(set1 | set2)

# Retrieve the top N most similar movies
def get_similar_movies(movie, n=5):
    similarities = []
    for other_movie in movie_to_genres.keys():
        if other_movie == movie:
            continue
        sim = jaccard_similarity(movie, other_movie)
        similarities.append((other_movie, sim))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [movie[0] for movie in similarities[:n]]

# Test the recommender system
input_movie = input("Enter the movie Name:")
n_recommendations = int(input("No.of movies of same genre :"))
print("Recommendations for", input_movie, ":")

#List to store the recommendations
recommendations_list = []

# Counter to keep track of the serial number
counter = 1

# Loop through the recommendations
for recommendation in get_similar_movies(input_movie, n_recommendations):
    recommendations_list.append((counter, recommendation))
    counter += 1

# Convert the list to a pandas DataFrame
recommendations_df = pd.DataFrame(recommendations_list, columns=["S.No", "Movie Title"])

# Print the DataFrame
recommendations_df



Enter the movie Name:Toy Story (1995)
No.of movies of same genre :5
Recommendations for Toy Story (1995) :


,S.No,Movie Title
0,1,Antz (1998)
1,2,Toy Story 2 (1999)
2,3,"Adventures of Rocky and Bullwinkle, The (2000)"
3,4,"Emperor's New Groove, The (2000)"
4,5,"Monsters, Inc. (2001)"


Create a collaborative based recommender system which recommends top N movies based on “K” similar users for a target user “u”

Example:

Input:

•UserID:1

•Num recommendations(N): 5

•Threshold for similar users (k) : 100

In [ ]:
import pandas as pd
import numpy as np

# Load the ratings data into a pandas dataframe
ratings_df = pd.read_csv('/content/drive/MyDrive/edureka/ratings.csv')

# Load the movie data into a pandas dataframe
movies_df = pd.read_csv('/content/drive/MyDrive/edureka/movies_csv_file_for _final_project.csv')

# Create a matrix of users and their ratings for each movie
user_movie_matrix = ratings_df.pivot_table(index="userId", columns="movieId", values="rating")

# Calculate the similarity between users
def pearson_similarity(user1, user2):
    user1_ratings = user_movie_matrix.loc[user1].dropna()
    user2_ratings = user_movie_matrix.loc[user2].dropna()
    common_movies = set(user1_ratings.index) & set(user2_ratings.index)
    if len(common_movies) == 0:
        return 0
    user1_mean = user1_ratings.mean()
    user2_mean = user2_ratings.mean()
    numerator = ((user1_ratings - user1_mean) * (user2_ratings - user2_mean)).sum()
    user1_denom = np.sqrt(((user1_ratings - user1_mean) ** 2).sum())
    user2_denom = np.sqrt(((user2_ratings - user2_mean) ** 2).sum())
    denominator = user1_denom * user2_denom
    return numerator / denominator

# Retrieve the top K most similar users
def get_similar_users(user, k=100):
    similarities = []
    for other_user in user_movie_matrix.index:
        if other_user == user:
            continue
        sim = pearson_similarity(user, other_user)
        similarities.append((other_user, sim))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [user[0] for user in similarities[:k]]

# Make recommendations to the target user
def make_recommendations(user, n=5, k=100):
    similar_users = get_similar_users(user, k)
    similar_users_ratings = user_movie_matrix.loc[similar_users]
    weighted_sum = similar_users_ratings.sum(axis=0) / np.abs(similar_users_ratings).sum(axis=0)
    recommendations = weighted_sum.sort_values(ascending=False).head(n).index
    movie_names = movies_df.set_index("movieId")["title"]
    return [(i+1, movie_names.loc[movie_id]) for i, movie_id in enumerate(recommendations)]

# Example usage
user = 1
n = 5
k = 100
recommendations = make_recommendations(user, n, k)

recommendations_list = []

# Loop through the recommendations
for i, recommendation in recommendations:
    recommendations_list.append((i, recommendation))

recommendations_df = pd.DataFrame(recommendations_list, columns=["S.No", "Movie Title"])

recommendations_df


<ipython-input-47-2b7ec99d41d1>:26: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator


,S.No,Movie Title
0,1,Toy Story (1995)
1,2,Tank (1984)
2,3,My Voyage to Italy (Il mio viaggio in Italia) ...
3,4,"Set-Up, The (1949)"
4,5,Gun Crazy (a.k.a. Deadly Is the Female) (1949)


Additional/Optional:
Createa GUI interface using Python libraries(ipywidgetsetc.)to play around with there commendation modules.


In [ ]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import ipywidgets as widgets


# Load the data
movies_df = pd.read_csv('/content/drive/MyDrive/edureka/movies_csv_file_for _final_project.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/edureka/ratings.csv')


# Preprocess the data
movies_with_ratings = pd.merge(movies, ratings, on="movieId")

# Create a dropdown widget to select the user
user_dropdown = widgets.Dropdown(options=ratings["userId"].unique(), value=1)

# Create a slider widget to select the number of recommendations
num_recs_slider = widgets.IntSlider(min=1, max=100, value=5)

# Create a function to get the recommendations
def get_recommendations(user_id, num_recs):
    # Get the movies rated by the user
    user_movies = movies_with_ratings[movies_with_ratings["userId"] == user_id]

    # Get the mean rating of each movie
    mean_ratings = user_movies.groupby("movieId")["rating"].mean()

    # Get the top N movies based on mean rating
    top_n = mean_ratings.sort_values(ascending=False).head(num_recs)

    # Get the movie titles
    recommendations = movies[movies["movieId"].isin(top_n.index)]["title"]

    return recommendations

# Display the widgets
widgets.VBox([user_dropdown, num_recs_slider])

# Display the recommendations
output = widgets.interact(get_recommendations, user_id=user_dropdown, num_recs=num_recs_slider)


interactive(children=(Dropdown(description='user_id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …